In [2]:
%matplotlib inline

from pathlib import Path

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df4 = pd.read_csv('dataset_v4.csv').set_index('building_id')

df1 = pd.read_csv('dataset_v1').set_index('building_id')


train_labels = pd.read_csv('train_labels.csv').set_index('building_id')

In [4]:
df1_cats = [
    'land_surface_condition',
    'foundation_type',
    'foundation_type',
    'roof_type',
    'ground_floor_type',
    'other_floor_type',
    'position',
    'plan_configuration',
    'legal_ownership_status'
]

for cat in df1_cats:
    df1[cat] = df1[cat].astype('category')

In [5]:
# df1 es mi set de entrenamiento
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 260601 entries, 802906 to 747594
Data columns (total 39 columns):
 #   Column                                  Non-Null Count   Dtype   
---  ------                                  --------------   -----   
 0   geo_level_1_id                          260601 non-null  int64   
 1   geo_level_2_id                          260601 non-null  int64   
 2   geo_level_3_id                          260601 non-null  int64   
 3   count_floors_pre_eq                     260601 non-null  int64   
 4   age                                     260601 non-null  int64   
 5   area_percentage                         260601 non-null  int64   
 6   height_percentage                       260601 non-null  int64   
 7   land_surface_condition                  260601 non-null  category
 8   foundation_type                         260601 non-null  category
 9   roof_type                               260601 non-null  category
 10  ground_floor_type          

In [32]:
from sklearn.model_selection import train_test_split
from catboost import Pool, CatBoostClassifier

In [36]:
x, y = df1.iloc[:,:-1], df1.iloc[:, -1]

Pruebo inicialmente entrenando con un subset del dataset de entrenamiento

In [37]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2048)

In [20]:
x_train.head()

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
building_id,,,,,,,,,,,,,,,,,,,,,
285985,3,1387,8564,5,80,10,10,t,r,n,...,False,False,False,False,False,False,False,False,False,False
673852,4,13,3967,3,20,6,5,t,r,q,...,False,False,False,False,False,False,False,False,False,False
18623,8,678,9322,3,20,9,6,t,r,n,...,False,False,False,False,False,False,False,False,False,False
115045,4,219,811,3,15,10,6,t,r,n,...,False,False,False,False,False,False,False,False,False,False
116269,20,174,5775,1,0,7,5,t,w,q,...,False,False,False,False,False,False,False,False,False,False


In [33]:
train_dataset = Pool(data=x_train,
                     label=y_train,
                     cat_features=df1_cats)

eval_dataset = Pool(data=x_test,
                    label=y_test,
                    cat_features=df1_cats)

catboost_model = CatBoostClassifier(
    learning_rate=1,
    depth=2,
    loss_function='MultiClass'
)
catboost_model.fit(train_dataset)
preds = catboost_model.predict(eval_dataset)

0:	learn: 0.8765715	total: 101ms	remaining: 1m 41s
1:	learn: 0.8219440	total: 213ms	remaining: 1m 46s
2:	learn: 0.7855552	total: 300ms	remaining: 1m 39s
3:	learn: 0.7696004	total: 386ms	remaining: 1m 36s
4:	learn: 0.7578900	total: 471ms	remaining: 1m 33s
5:	learn: 0.7519191	total: 563ms	remaining: 1m 33s
6:	learn: 0.7479196	total: 668ms	remaining: 1m 34s
7:	learn: 0.7454358	total: 759ms	remaining: 1m 34s
8:	learn: 0.7410901	total: 851ms	remaining: 1m 33s
9:	learn: 0.7353147	total: 937ms	remaining: 1m 32s
10:	learn: 0.7325485	total: 1.02s	remaining: 1m 32s
11:	learn: 0.7293705	total: 1.11s	remaining: 1m 31s
12:	learn: 0.7251032	total: 1.21s	remaining: 1m 32s
13:	learn: 0.7227719	total: 1.3s	remaining: 1m 31s
14:	learn: 0.7208490	total: 1.4s	remaining: 1m 31s
15:	learn: 0.7195869	total: 1.49s	remaining: 1m 31s
16:	learn: 0.7187973	total: 1.58s	remaining: 1m 31s
17:	learn: 0.7177087	total: 1.69s	remaining: 1m 31s
18:	learn: 0.7159722	total: 1.78s	remaining: 1m 32s
19:	learn: 0.7139108	tot

159:	learn: 0.6665988	total: 14.4s	remaining: 1m 15s
160:	learn: 0.6663728	total: 14.5s	remaining: 1m 15s
161:	learn: 0.6662899	total: 14.6s	remaining: 1m 15s
162:	learn: 0.6660518	total: 14.7s	remaining: 1m 15s
163:	learn: 0.6659009	total: 14.8s	remaining: 1m 15s
164:	learn: 0.6657528	total: 14.9s	remaining: 1m 15s
165:	learn: 0.6656458	total: 15s	remaining: 1m 15s
166:	learn: 0.6655193	total: 15s	remaining: 1m 15s
167:	learn: 0.6653481	total: 15.1s	remaining: 1m 14s
168:	learn: 0.6652751	total: 15.2s	remaining: 1m 14s
169:	learn: 0.6652162	total: 15.3s	remaining: 1m 14s
170:	learn: 0.6651952	total: 15.4s	remaining: 1m 14s
171:	learn: 0.6650573	total: 15.5s	remaining: 1m 14s
172:	learn: 0.6649269	total: 15.6s	remaining: 1m 14s
173:	learn: 0.6647990	total: 15.7s	remaining: 1m 14s
174:	learn: 0.6647641	total: 15.8s	remaining: 1m 14s
175:	learn: 0.6646498	total: 15.9s	remaining: 1m 14s
176:	learn: 0.6644519	total: 15.9s	remaining: 1m 14s
177:	learn: 0.6641536	total: 16s	remaining: 1m 14s

318:	learn: 0.6526482	total: 28.6s	remaining: 1m 1s
319:	learn: 0.6524183	total: 28.7s	remaining: 1m
320:	learn: 0.6524152	total: 28.8s	remaining: 1m
321:	learn: 0.6523414	total: 28.9s	remaining: 1m
322:	learn: 0.6522608	total: 28.9s	remaining: 1m
323:	learn: 0.6522511	total: 29s	remaining: 1m
324:	learn: 0.6522490	total: 29.1s	remaining: 1m
325:	learn: 0.6522153	total: 29.2s	remaining: 1m
326:	learn: 0.6521404	total: 29.3s	remaining: 1m
327:	learn: 0.6519892	total: 29.4s	remaining: 1m
328:	learn: 0.6519088	total: 29.5s	remaining: 1m
329:	learn: 0.6518594	total: 29.6s	remaining: 1m
330:	learn: 0.6518162	total: 29.7s	remaining: 59.9s
331:	learn: 0.6517685	total: 29.7s	remaining: 59.8s
332:	learn: 0.6517651	total: 29.8s	remaining: 59.7s
333:	learn: 0.6517309	total: 29.9s	remaining: 59.7s
334:	learn: 0.6517192	total: 30s	remaining: 59.6s
335:	learn: 0.6516345	total: 30.1s	remaining: 59.5s
336:	learn: 0.6515083	total: 30.2s	remaining: 59.4s
337:	learn: 0.6513226	total: 30.3s	remaining: 59.

480:	learn: 0.6454809	total: 43s	remaining: 46.4s
481:	learn: 0.6454535	total: 43.1s	remaining: 46.3s
482:	learn: 0.6454185	total: 43.2s	remaining: 46.2s
483:	learn: 0.6454107	total: 43.3s	remaining: 46.1s
484:	learn: 0.6454038	total: 43.3s	remaining: 46s
485:	learn: 0.6454007	total: 43.4s	remaining: 45.9s
486:	learn: 0.6453982	total: 43.5s	remaining: 45.8s
487:	learn: 0.6453583	total: 43.6s	remaining: 45.7s
488:	learn: 0.6452646	total: 43.7s	remaining: 45.7s
489:	learn: 0.6452493	total: 43.8s	remaining: 45.6s
490:	learn: 0.6451901	total: 43.9s	remaining: 45.5s
491:	learn: 0.6451617	total: 44s	remaining: 45.4s
492:	learn: 0.6451544	total: 44.1s	remaining: 45.3s
493:	learn: 0.6451181	total: 44.1s	remaining: 45.2s
494:	learn: 0.6451082	total: 44.2s	remaining: 45.1s
495:	learn: 0.6450128	total: 44.3s	remaining: 45s
496:	learn: 0.6449914	total: 44.4s	remaining: 44.9s
497:	learn: 0.6449804	total: 44.5s	remaining: 44.8s
498:	learn: 0.6449373	total: 44.6s	remaining: 44.7s
499:	learn: 0.644929

639:	learn: 0.6400886	total: 57.1s	remaining: 32.1s
640:	learn: 0.6400772	total: 57.2s	remaining: 32s
641:	learn: 0.6400756	total: 57.3s	remaining: 31.9s
642:	learn: 0.6400452	total: 57.4s	remaining: 31.8s
643:	learn: 0.6400165	total: 57.4s	remaining: 31.8s
644:	learn: 0.6400105	total: 57.5s	remaining: 31.7s
645:	learn: 0.6399931	total: 57.6s	remaining: 31.6s
646:	learn: 0.6398714	total: 57.7s	remaining: 31.5s
647:	learn: 0.6398337	total: 57.8s	remaining: 31.4s
648:	learn: 0.6398293	total: 57.9s	remaining: 31.3s
649:	learn: 0.6398061	total: 58s	remaining: 31.2s
650:	learn: 0.6396768	total: 58.1s	remaining: 31.1s
651:	learn: 0.6396591	total: 58.1s	remaining: 31s
652:	learn: 0.6396561	total: 58.2s	remaining: 30.9s
653:	learn: 0.6396366	total: 58.3s	remaining: 30.9s
654:	learn: 0.6396041	total: 58.4s	remaining: 30.8s
655:	learn: 0.6395648	total: 58.5s	remaining: 30.7s
656:	learn: 0.6395232	total: 58.6s	remaining: 30.6s
657:	learn: 0.6394846	total: 58.7s	remaining: 30.5s
658:	learn: 0.6394

800:	learn: 0.6354857	total: 1m 11s	remaining: 17.7s
801:	learn: 0.6354787	total: 1m 11s	remaining: 17.6s
802:	learn: 0.6353807	total: 1m 11s	remaining: 17.6s
803:	learn: 0.6353620	total: 1m 11s	remaining: 17.5s
804:	learn: 0.6353314	total: 1m 11s	remaining: 17.4s
805:	learn: 0.6353166	total: 1m 11s	remaining: 17.3s
806:	learn: 0.6353036	total: 1m 11s	remaining: 17.2s
807:	learn: 0.6352724	total: 1m 12s	remaining: 17.1s
808:	learn: 0.6352680	total: 1m 12s	remaining: 17s
809:	learn: 0.6352670	total: 1m 12s	remaining: 16.9s
810:	learn: 0.6352502	total: 1m 12s	remaining: 16.9s
811:	learn: 0.6351584	total: 1m 12s	remaining: 16.8s
812:	learn: 0.6351428	total: 1m 12s	remaining: 16.7s
813:	learn: 0.6351208	total: 1m 12s	remaining: 16.6s
814:	learn: 0.6350371	total: 1m 12s	remaining: 16.5s
815:	learn: 0.6350336	total: 1m 12s	remaining: 16.4s
816:	learn: 0.6350105	total: 1m 12s	remaining: 16.3s
817:	learn: 0.6350104	total: 1m 12s	remaining: 16.2s
818:	learn: 0.6349975	total: 1m 13s	remaining: 1

956:	learn: 0.6318131	total: 1m 25s	remaining: 3.84s
957:	learn: 0.6317436	total: 1m 25s	remaining: 3.75s
958:	learn: 0.6317358	total: 1m 25s	remaining: 3.66s
959:	learn: 0.6317315	total: 1m 25s	remaining: 3.57s
960:	learn: 0.6316976	total: 1m 25s	remaining: 3.48s
961:	learn: 0.6316958	total: 1m 25s	remaining: 3.39s
962:	learn: 0.6316787	total: 1m 25s	remaining: 3.3s
963:	learn: 0.6316612	total: 1m 26s	remaining: 3.21s
964:	learn: 0.6316574	total: 1m 26s	remaining: 3.13s
965:	learn: 0.6316536	total: 1m 26s	remaining: 3.04s
966:	learn: 0.6316328	total: 1m 26s	remaining: 2.95s
967:	learn: 0.6316313	total: 1m 26s	remaining: 2.86s
968:	learn: 0.6316192	total: 1m 26s	remaining: 2.77s
969:	learn: 0.6316145	total: 1m 26s	remaining: 2.68s
970:	learn: 0.6315963	total: 1m 26s	remaining: 2.59s
971:	learn: 0.6315689	total: 1m 26s	remaining: 2.5s
972:	learn: 0.6315678	total: 1m 26s	remaining: 2.41s
973:	learn: 0.6315403	total: 1m 26s	remaining: 2.32s
974:	learn: 0.6315221	total: 1m 27s	remaining: 2

In [38]:
from sklearn.metrics import f1_score

f1_score(y_test, preds, average='micro')

0.7095412597609408

Ahora pruebo con el dataset de entrenamiento completo

In [41]:
test_values = pd.read_csv('encoded_test_values.csv', index_col='building_id')

In [49]:
train_dataset = Pool(data=x,
                     label=y,
                     cat_features=df1_cats)

catboost_model = CatBoostClassifier(
    learning_rate=0.3,
    depth=5,
    loss_function='MultiClass'
)
catboost_model.fit(train_dataset, verbose=False)
preds = catboost_model.predict(train_dataset)
f1_score(y, preds, average='micro')

0.7446325992609392

In [55]:
test_values = pd.read_csv('test_values.csv').set_index('building_id')

for cat in df1_cats:
    test_values[cat] = test_values[cat].astype('category')
test_values.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 86868 entries, 300051 to 501372
Data columns (total 38 columns):
 #   Column                                  Non-Null Count  Dtype   
---  ------                                  --------------  -----   
 0   geo_level_1_id                          86868 non-null  int64   
 1   geo_level_2_id                          86868 non-null  int64   
 2   geo_level_3_id                          86868 non-null  int64   
 3   count_floors_pre_eq                     86868 non-null  int64   
 4   age                                     86868 non-null  int64   
 5   area_percentage                         86868 non-null  int64   
 6   height_percentage                       86868 non-null  int64   
 7   land_surface_condition                  86868 non-null  category
 8   foundation_type                         86868 non-null  category
 9   roof_type                               86868 non-null  category
 10  ground_floor_type                       

In [64]:
test_dataset = Pool(data=test_values,
                    cat_features=df1_cats)

predictions = catboost_model.predict(test_dataset)

In [65]:
sub_format = pd.read_csv('submission_format.csv', index_col='building_id')

In [67]:
submission = pd.DataFrame(data=predictions, columns=sub_format.columns, index=sub_format.index)
submission.to_csv('submission_catboost.csv')